In [15]:
#TODO: CHANGE THIS BASED ON YOUR OWN LOCAL SETTINGS
#MY_HOME_ABS_PATH = "/Users/jetcalz07/Desktop/MIDS/W210_Capstone/co2-flux-hourly-gpp-modeling"
MY_HOME_ABS_PATH = "/root/co2-flux-hourly-gpp-modeling"

In [10]:
from tqdm import tqdm
from tempfile import TemporaryFile
import pandas as pd
import csv
import os
os.chdir(MY_HOME_ABS_PATH)
import sys
sys.path.append(os.path.abspath("./code/src/tools"))
from CloudIO.AzStorageClient import AzStorageClient

## Define Constants

In [11]:
root_dir =  MY_HOME_ABS_PATH
tmp_dir =  root_dir + os.sep + '.tmp'
cred_dir = root_dir + os.sep + '.cred'
az_cred_file = cred_dir + os.sep + 'azblobcred.json'

if not (os.path.exists(tmp_dir)):
    print("Making tmp_dir")
    os.mkdir(tmp_dir)

In [41]:
# Initialize Azure Storage Client
azStorageClient = AzStorageClient(az_cred_file)
container_name = 'full-half-hourly-raw'
blob_name_list = azStorageClient.listBlobs(container_name)

## Pull Site Data

In [13]:
# Check one example to get colnames
ex_file = tmp_dir + os.sep + 'site_example_ZM-Mon.csv' 
site_ex = pd.read_csv(ex_file)
col_names = site_ex.columns
site_ex.head()

,TIMESTAMP_START,TIMESTAMP_END,TA_F,TA_F_QC,TA_ERA,SW_IN_POT,SW_IN_F,SW_IN_F_QC,SW_IN_ERA,LW_IN_F,...,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen
0,200003020000,200003020030,19.618,0,19.504,0.0,0.000,0,0.0,400.52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DBF,Temperate
1,200003020030,200003020100,19.800,0,19.379,0.0,0.000,0,0.0,406.29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DBF,Temperate
2,200003020100,200003020130,19.681,0,19.254,0.0,0.318,0,0.0,408.51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DBF,Temperate
3,200003020130,200003020200,18.311,0,19.130,0.0,0.224,0,0.0,403.84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DBF,Temperate
4,200003020200,200003020230,17.640,0,19.005,0.0,0.789,0,0.0,399.84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DBF,Temperate


In [42]:
# Download all HH site file to .tmp dir
for i, blob in tqdm(enumerate(blob_name_list)):
    blob_name = blob.name
    print(blob_name)
    data = azStorageClient.downloadBlob(container_name, blob_name)
    
    # Create a temporary file to write the CSV file with column names
    with TemporaryFile(mode='w+', newline='') as tmp_file:
        writer = csv.writer(tmp_file)
        
        # check if the csv has the column names, if not add them in
        data_lines = data.decode().splitlines()
        reader = csv.reader(data_lines)
        header = next(reader)

        # Check if the first row contains the expected columns
        if tuple(header) == tuple(col_names):
            # If the first row matches col_names, write it as the header
            writer.writerow(col_names)
            data_start_idx = 1
        else:
            # If the first row does not match col_names, assume it is data and write it as is
            writer.writerow(col_names)
            data_start_idx = 0
        
        # Write the data as the remaining rows
        writer.writerows(csv.reader(data_lines[data_start_idx:]))
        
        # Save the temporary file to disk
        filename = os.path.join(tmp_dir, blob_name)
        with open(filename, "w", newline='') as f:
            tmp_file.seek(0)
            f.write(tmp_file.read())
            
    if i > 2:
        break

0it [00:00, ?it/s]

data_full_half_hourly_raw_v0_1_AR-SLu.csv
Missing cols


1it [00:01,  1.96s/it]

data_full_half_hourly_raw_v0_1_AR-Vir.csv
Missing cols


2it [00:03,  1.67s/it]

data_full_half_hourly_raw_v0_1_AT-Neu.csv


3it [00:07,  2.96s/it]

data_full_half_hourly_raw_v0_1_AU-ASM.csv
Missing cols


3it [00:11,  3.71s/it]


In [43]:
# Check that the Azure df == the locally uploaded example from GDrive
#local_filename = tmp_dir + os.sep + f'data_full_half_hourly_raw_v0_1_ZM-Mon.csv'  # <--- addd in at end to compare
local_filename = tmp_dir + os.sep + f'data_full_half_hourly_raw_v0_1_AT-Neu.csv' 
site_df = pd.read_csv(local_filename)
site_df.head()
#pd.equals(site_df, site_ex)

/tmp/ipykernel_585/3765866218.py:4: DtypeWarning: Columns (33,38,39,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  site_df = pd.read_csv(local_filename)


,TIMESTAMP_START,TIMESTAMP_END,TA_F,TA_F_QC,TA_ERA,SW_IN_POT,SW_IN_F,SW_IN_F_QC,SW_IN_ERA,LW_IN_F,...,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen
0,0,2.611260e+02,1.452,0,1.020,0.0000,0.000,0.0,91.300,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,200704270500,2.007043e+11,5.299,0,7.552,20.0592,7.224,0.0,4.647,265.12,...,0.186405,0.0386,0.2534,0.0213,0.0482,0.2987,0.1844,0.0906,GRA,Cold
2,200704270530,2.007043e+11,5.732,0,7.613,126.0630,25.806,0.0,48.229,274.22,...,0.186405,0.0386,0.2534,0.0213,0.0482,0.2987,0.1844,0.0906,GRA,Cold
3,200704270600,2.007043e+11,5.955,0,7.674,243.2710,48.491,0.0,92.497,271.35,...,0.186405,0.0386,0.2534,0.0213,0.0482,0.2987,0.1844,0.0906,GRA,Cold
4,200704270630,2.007043e+11,6.861,0,7.734,360.3130,63.257,0.0,136.694,262.21,...,0.186405,0.0386,0.2534,0.0213,0.0482,0.2987,0.1844,0.0906,GRA,Cold


In [ ]:
tmp_dir_size = len([x for x in os.listdir(tmp_dir) if 'data_full_half_hourly_raw' in x])
print(f"Number of site files in tmp_dir: {tmp_dir_size}")

In [23]:
some_files = [x for x in os.listdir(tmp_dir) if 'data_full_half_hourly_raw' in x][:5]
for file in some_files:
    site_df = pd.read_csv(tmp_dir + os.sep + file)
    display(site_df.head(3))
    

/tmp/ipykernel_585/3499514034.py:3: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,44,45,46,47,50,51,52,53,54,55,56,57,58,59,60,61) have mixed types. Specify dtype option on import or set low_memory=False.
  site_df = pd.read_csv(tmp_dir + os.sep + file)


,TIMESTAMP_START,TIMESTAMP_END,TA_F,TA_F_QC,TA_ERA,SW_IN_POT,SW_IN_F,SW_IN_F_QC,SW_IN_ERA,LW_IN_F,...,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen
0,TIMESTAMP_START,TIMESTAMP_END,TA_F,TA_F_QC,TA_ERA,SW_IN_POT,SW_IN_F,SW_IN_F_QC,SW_IN_ERA,LW_IN_F,...,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen
1,201608130000,201608130030,21.71,0,16.076,0.0,0.0,0,0.0,324.962,...,0.2573538727641106,0.0594,0.3593,0.034,0.0691,0.3277,0.2316,0.1107,CRO,Temperate
2,201608130030,201608130100,20.3,0,15.763,0.0,0.0,0,0.0,324.962,...,0.2573538727641106,0.0594,0.3593,0.034,0.0691,0.3277,0.2316,0.1107,CRO,Temperate


/tmp/ipykernel_585/3499514034.py:3: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,44,45,46,47,50,51,52,53,54,55,56,57,58,59,60,61) have mixed types. Specify dtype option on import or set low_memory=False.
  site_df = pd.read_csv(tmp_dir + os.sep + file)


,TIMESTAMP_START,TIMESTAMP_END,TA_F,TA_F_QC,TA_ERA,SW_IN_POT,SW_IN_F,SW_IN_F_QC,SW_IN_ERA,LW_IN_F,...,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen
0,TIMESTAMP_START,TIMESTAMP_END,TA_F,TA_F_QC,TA_ERA,SW_IN_POT,SW_IN_F,SW_IN_F_QC,SW_IN_ERA,LW_IN_F,...,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen
1,200909030000,200909030030,14.25,0,14.143,0.0,0.0,0,0.0,365.277,...,0.1431338943243026,0.0261,0.189,0.0222,0.0325,0.1697,0.0835,0.0356,ENF,Cold
2,200909030030,200909030100,14.21,0,14.087,0.0,0.0,0,0.0,365.277,...,0.1431338943243026,0.0261,0.189,0.0222,0.0325,0.1697,0.0835,0.0356,ENF,Cold


/tmp/ipykernel_585/3499514034.py:3: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,44,45,46,47,50,51,52,53,54,55,56,57,58,59,60,61) have mixed types. Specify dtype option on import or set low_memory=False.
  site_df = pd.read_csv(tmp_dir + os.sep + file)


,TIMESTAMP_START,TIMESTAMP_END,TA_F,TA_F_QC,TA_ERA,SW_IN_POT,SW_IN_F,SW_IN_F_QC,SW_IN_ERA,LW_IN_F,...,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen
0,TIMESTAMP_START,TIMESTAMP_END,TA_F,TA_F_QC,TA_ERA,SW_IN_POT,SW_IN_F,SW_IN_F_QC,SW_IN_ERA,LW_IN_F,...,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen
1,200401140000,200401140030,-6.835,0,-9.649,0.0,0.0,0,0.0,241.6,...,0.0395064935064935,0.1664,0.234,0.0761,0.1248,0.301,0.3032,0.2621,GRA,Arid
2,200401140030,200401140100,-6.9,0,-10.349,0.0,0.0,0,0.0,242.2,...,0.0395064935064935,0.1664,0.234,0.0761,0.1248,0.301,0.3032,0.2621,GRA,Arid


/tmp/ipykernel_585/3499514034.py:3: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,44,45,46,47,50,51,52,53,54,55,56,57,58,59,60,61) have mixed types. Specify dtype option on import or set low_memory=False.
  site_df = pd.read_csv(tmp_dir + os.sep + file)


,TIMESTAMP_START,TIMESTAMP_END,TA_F,TA_F_QC,TA_ERA,SW_IN_POT,SW_IN_F,SW_IN_F_QC,SW_IN_ERA,LW_IN_F,...,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen
0,TIMESTAMP_START,TIMESTAMP_END,TA_F,TA_F_QC,TA_ERA,SW_IN_POT,SW_IN_F,SW_IN_F_QC,SW_IN_ERA,LW_IN_F,...,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen
1,201002130000,201002130030,26.32,0,25.237,0.0,0.0,0,0.0,403.688,...,0.3244497714575927,0.0241,0.3697,0.0148,0.0555,0.2788,0.1235,0.046,ENF,Temperate
2,201002130030,201002130100,26.65,0,25.115,0.0,0.0,0,0.0,403.688,...,0.3244497714575927,0.0241,0.3697,0.0148,0.0555,0.2788,0.1235,0.046,ENF,Temperate


/tmp/ipykernel_585/3499514034.py:3: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,44,45,46,47,50,51,52,53,54,55,56,57,58,59,60,61) have mixed types. Specify dtype option on import or set low_memory=False.
  site_df = pd.read_csv(tmp_dir + os.sep + file)


,TIMESTAMP_START,TIMESTAMP_END,TA_F,TA_F_QC,TA_ERA,SW_IN_POT,SW_IN_F,SW_IN_F_QC,SW_IN_ERA,LW_IN_F,...,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen
0,TIMESTAMP_START,TIMESTAMP_END,TA_F,TA_F_QC,TA_ERA,SW_IN_POT,SW_IN_F,SW_IN_F_QC,SW_IN_ERA,LW_IN_F,...,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen
1,201410030000,201410030030,9.25,0,9.959,0.0,0.0,0,0.0,262.764,...,0.0304307663551401,0.1082,0.1593,0.059,0.0843,0.1993,0.2122,0.1493,OSH,Arid
2,201410030030,201410030100,8.86,0,9.803,0.0,0.0,0,0.0,262.764,...,0.0304307663551401,0.1082,0.1593,0.059,0.0843,0.1993,0.2122,0.1493,OSH,Arid
